In [20]:
from mongoConnection import MongoConnection
import pandas as pd
import pprint


db = MongoConnection()
ma_list = [10,50]
max_drawdown = 0
win_loss = 0
exp_profit = 0 


In [22]:
 #This returns an iterable 'cursor' object
res = db.query_from_to(1638331200,1641196800)

df = pd.DataFrame(res)
df.rename(columns = {'x':'Time'}, inplace = True)
split = pd.DataFrame(df["y"].tolist(),columns=["Open","High","Low","Close"])
df = pd.concat([df,split], axis=1)
df = df.drop("y", axis=1)

print(df)



           Time         Open         High          Low        Close
0    1638331200  4632.747295  4771.598038  4608.797507  4757.090486
1    1638345600  4763.100801  4782.300037  4692.148759  4770.009313
2    1638360000  4771.247355  4771.247355  4660.329251  4671.153844
3    1638374400  4677.132670  4764.705298  4664.995665  4729.167125
4    1638388800  4730.074852  4760.600238  4601.475675  4662.434861
..          ...          ...          ...          ...          ...
195  1641139200  3744.774385  3787.147237  3732.015065  3778.575584
196  1641153600  3779.027935  3852.229565  3774.830568  3800.581917
197  1641168000  3799.548338  3840.500713  3796.804239  3832.219825
198  1641182400  3829.323196  3829.323196  3778.568244  3786.349383
199  1641196800  3785.820854  3822.035001  3783.154087  3809.373702

[200 rows x 5 columns]


In [37]:
def is_trade(row):
    if row.DIFF >= 0 and row.DIFF_PREV < 0:
        return 1
    if row.DIFF <= 0 and row.DIFF_PREV > 0:
        return -1
    return 0

def max_draw(row):
    local_max = 0
    max_drawdown = 0
    for num in row:
        if num < 0:
            local_max+=num
            if local_max < max_drawdown:
                max_drawdown = local_max
        else:
            local_max = 0
    return max_drawdown

def winCount(row):
    count = 0
    for num in row:
        if num >= 0:
            count+=1
    return count
            
    
            

In [24]:
# These are the fast and slow moving averages that we are applying

df_ma = df.copy()
for ma in ma_list:
    df_ma[f'MA_{ma}'] = df_ma.Close.rolling(window=ma).mean() #Create the moving average column
    
df_ma['DIFF'] = df_ma.MA_10 - df_ma.MA_50 # create a new column that takes the different between the two moving averages
df_ma['DIFF_PREV'] = df_ma.DIFF.shift(1) # creates a new column that shifts down the diff column, used to determine if there will be a trade
df_ma['IS_TRADE'] = df_ma.apply(is_trade, axis=1) #Lets you know if you should've made trade based on strategy.

df_trades = df_ma[df_ma.IS_TRADE!=0].copy() # Creates a dataframe to analyzes all entries that you should've traded... for later stats 

df_ma.tail()


,Time,Open,High,Low,Close,MA_10,MA_50,DIFF,DIFF_PREV,IS_TRADE
195,1641139200,3744.774385,3787.147237,3732.015065,3778.575584,3740.844695,3857.438345,-116.593650,-132.467956,0
196,1641153600,3779.027935,3852.229565,3774.830568,3800.581917,3748.386603,3852.382690,-103.996087,-116.593650,0
197,1641168000,3799.548338,3840.500713,3796.804239,3832.219825,3759.784747,3847.411658,-87.626911,-103.996087,0
198,1641182400,3829.323196,3829.323196,3778.568244,3786.349383,3768.764246,3841.581892,-72.817645,-87.626911,0
199,1641196800,3785.820854,3822.035001,3783.154087,3809.373702,3777.163102,3835.868123,-58.705021,-72.817645,0


In [39]:

df_trades["DELTA"] = df_trades.Close.diff().shift(-1)
df_trades["GAIN"] = df_trades["DELTA"] * df_trades["IS_TRADE"]
exp_profit = df_trades["GAIN"].sum() # tested... works

max_drawdown = max_draw(df_trades["GAIN"]) #tested... works 

print(exp_profit)
print(max_drawdown)

win_loss = winCount(df_trades["GAIN"]) / (len(df_trades)-1)
print(win_loss)

print(df_trades)

-445.3822197636996
-589.4033359737882
0.16666666666666666
           Time         Open         High          Low        Close  \
55   1639123200  4142.564182  4164.916009  4027.594265  4027.785248   
97   1639728000  3968.853222  3987.220756  3857.281084  3883.764132   
99   1639756800  3835.512395  3850.415086  3707.122902  3836.906248   
111  1639929600  3927.438698  3996.114480  3902.388136  3925.594058   
116  1640001600  3825.376206  3834.582297  3769.261334  3812.354219   
122  1640088000  4003.359353  4033.740537  3995.781576  4026.637679   
165  1640707200  3922.220768  3932.877725  3868.429209  3900.303336   

           MA_10        MA_50       DIFF  DIFF_PREV  IS_TRADE       DELTA  \
55   4251.607527  4277.038882 -25.431355   1.062176        -1 -144.021116   
97   4000.112294  3996.086709   4.025585 -28.747449         1  -46.857884   
99   3967.702584  3974.374451  -6.671867   0.547756        -1   88.687810   
111  3940.950976  3937.060258   3.890719  -2.272403         1 -11